## Prequisites and Preprocessing

### Permissions & environment variable

In [6]:
from sagemaker import get_execution_role

role = get_execution_role()
bucket='beichuan'

### Setup

In [8]:
from sagemaker import get_execution_role

#Bucket location to save your custom code in tar.gz format.
custom_code_upload_location = 's3://beichuan/customcode/mxnet'

#Bucket location where results of model training are saved.
model_artifacts_location = 's3://beichuan/artifacts'

#IAM execution role that gives SageMaker access to resources in your AWS account.
#We can use the SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

### The training script

In [9]:
!cat mnist.py

import logging

import gzip
import mxnet as mx
import numpy as np
import os
import struct


def load_data(path):
    with gzip.open(find_file(path, "labels.gz")) as flbl:
        struct.unpack(">II", flbl.read(8))
        labels = np.fromstring(flbl.read(), dtype=np.int8)
    with gzip.open(find_file(path, "images.gz")) as fimg:
        _, _, rows, cols = struct.unpack(">IIII", fimg.read(16))
        images = np.fromstring(fimg.read(), dtype=np.uint8).reshape(len(labels), rows, cols)
        images = images.reshape(images.shape[0], 1, 28, 28).astype(np.float32) / 255
    return labels, images


def find_file(root_path, file_name):
    for root, dirs, files in os.walk(root_path):
        if file_name in files:
            return os.path.join(root, file_name)


def build_graph():
    data = mx.sym.var('data')
    data = mx.sym.flatten(data=data)
    fc1 = mx.sym.FullyConnected(data=data, num_hidden=128)
    act1 = mx.sym.Activation(data=fc1, act_type="relu")
    fc2 = mx.sym.FullyConnect

### SageMaker's MXNet estimator class

In [10]:
from sagemaker.mxnet import MXNet

mnist_estimator = MXNet(entry_point='mnist.py',
                        role=role,
                        output_path=model_artifacts_location,
                        code_location=custom_code_upload_location,
                        train_instance_count=1, 
                        train_instance_type='ml.m4.xlarge',
                        hyperparameters={'learning_rate': 0.1})

### Running the Training Job

In [11]:
%%time
import boto3

region = boto3.Session().region_name
train_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/train'.format(region)
test_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/test'.format(region)

mnist_estimator.fit({'train': train_data_location, 'test': test_data_location})

INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-py2-cpu-2018-03-08-05-00-57-269


......................................................................
executing startup script (first run)
2018-03-08 05:06:14,447 INFO - root - running container entrypoint
2018-03-08 05:06:14,447 INFO - root - starting train task
2018-03-08 05:06:15,674 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'enable_cloudwatch_metrics': False, 'available_gpus': 0, 'channels': {u'test': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}, u'train': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}}, '_ps_verbose': 0, 'resource_config': {u'current_host': u'algo-1', u'hosts': [u'algo-1']}, 'user_script_name': u'mnist.py', 'input_config_dir': '/opt/ml/input/config', 'channel_dirs': {u'test': u'/opt/ml/input/data/test', u'train': u'/opt/ml/input/data/train'}, 'code_dir': '/opt/ml/code', 'output_data_dir': '/opt/ml/output/data/', 'output_dir': '/opt/ml/output', 'model_dir': '/op

2018-03-08 05:07:01,763 INFO - root - Epoch[23] Train-accuracy=0.997475
2018-03-08 05:07:01,763 INFO - root - Epoch[23] Time cost=1.997
2018-03-08 05:07:01,903 INFO - root - Epoch[23] Validation-accuracy=0.976000
2018-03-08 05:07:02,314 INFO - root - Epoch[24] Batch [100]#011Speed: 24464.11 samples/sec#011accuracy=0.997921
2018-03-08 05:07:02,696 INFO - root - Epoch[24] Batch [200]#011Speed: 26227.51 samples/sec#011accuracy=0.998100
2018-03-08 05:07:03,025 INFO - root - Epoch[24] Batch [300]#011Speed: 30355.37 samples/sec#011accuracy=0.998500
2018-03-08 05:07:03,388 INFO - root - Epoch[24] Batch [400]#011Speed: 27597.63 samples/sec#011accuracy=0.997600
2018-03-08 05:07:03,719 INFO - root - Epoch[24] Batch [500]#011Speed: 30209.73 samples/sec#011accuracy=0.998600
2018-03-08 05:07:04,059 INFO - root - Epoch[24] Train-accuracy=0.998081
2018-03-08 05:07:04,059 INFO - root - Epoch[24] Time cost=2.156
2018-03-08 05:07:04,244 INFO - root - Epoch[24] Validation-accuracy=0.976200


ValueError: Error training sagemaker-mxnet-py2-cpu-2018-03-08-05-00-57-269: Failed Reason: ClientError: Model artifact upload failed:Error 7: Supplied bucket is not in the region associated with the training job (us-east-1)

### Creating an inference Endpoint

In [ ]:
%%time

predictor = mnist_estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

### Making an inference request

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In [ ]:
response = predictor.predict(data)
print('Raw prediction result:')
print(response)

labeled_predictions = list(zip(range(10), response[0]))
print('Labeled predictions: ')
print(labeled_predictions)

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print('Most likely answer: {}'.format(labeled_predictions[0]))

Reference: 
* 